In [5]:
import pandas as pd
import plotly.express as px
from datetime import datetime


df = pd.read_csv("play store data.csv")


df["Installs"] = df["Installs"].fillna("0")


df["Installs"] = df["Installs"].astype(str).str.replace(r"[^\d]", "", regex=True)


df["Installs"] = pd.to_numeric(df["Installs"], errors="coerce").fillna(0).astype(int)


df["Last Updated"] = df["Last Updated"].astype(str)




df = df[df["Last Updated"].str.contains(r"\d{4}", na=False)]


df["Last Updated"] = pd.to_datetime(df["Last Updated"], errors="coerce")


df = df.dropna(subset=["Last Updated"])


df = df[
    (df["Content Rating"] == "Teen") &
    (df["App"].str.startswith("E")) &
    (df["Installs"] > 10_000)
]

df["Month"] = df["Last Updated"].dt.to_period("M")
df_grouped = df.groupby(["Month", "Category"])["Installs"].sum().reset_index()


df_grouped["Previous Month Installs"] = df_grouped.groupby("Category")["Installs"].shift(1)
df_grouped["Growth Rate"] = ((df_grouped["Installs"] - df_grouped["Previous Month Installs"]) /
                             df_grouped["Previous Month Installs"]) * 100


df_grouped["Significant Growth"] = df_grouped["Growth Rate"] > 20
df_grouped["Month"] = df_grouped["Month"].astype(str) 


current_time = datetime.now().time()
start_time = datetime.strptime("18:00:00", "%H:%M:%S").time()
end_time = datetime.strptime("21:00:00", "%H:%M:%S").time()

if start_time <= current_time <= end_time:
    fig = px.line(
        df_grouped,
        x="Month",
        y="Installs",
        color="Category",
        title="Total Installs Trend Over Time (Segmented by Category)",
        labels={"Month": "Month", "Installs": "Total Installs", "Category": "App Category"},
        line_group="Category"
    )

    for category in df_grouped["Category"].unique():
        df_cat = df_grouped[df_grouped["Category"] == category]
        fig.add_scatter(
            x=df_cat.loc[df_cat["Significant Growth"], "Month"],
            y=df_cat.loc[df_cat["Significant Growth"], "Installs"],
            fill="tozeroy",
            mode="none",
            fillcolor="rgba(255, 0, 0, 0.2)",
            name=f"Significant Growth ({category})"
        )

    fig.show()
else:
    print(" graph  works only between 6 PM IST to 9 PM .")


 graph  works only between 6 PM IST to 9 PM .
